# Loading code

In [84]:
import pdfplumber
import pandas as pd

tables = []

with pdfplumber.open("mvfodevarer.pdf") as pdf:
    # Iterate through each page
    for page in pdf.pages:
        # Extract tables from the page
        page_tables = page.extract_tables({
           'vertical_strategy': 'lines',
           'horizontal_strategy': 'lines',
           'intersection_x_tolerance': 8,
           'intersection_y_tolerance': 8
        })
 

        if page_tables:
            for table in page_tables:
                if table:
                    tables.append({
                        'page': pdf.pages.index(page) + 1,
                        'data': table
                    })


# for i, item in enumerate(tables[0]):
#     #Detect new 
#     #
#     #
#     #
#     #

In [85]:
len(tables)

38

In [86]:


# for table in tables:
#     for row in table['data']:    
#         if row.__contains__('Madvare'):
#             print(f"Page: {table['page']}")
#             print(row)
        
tables[0]['data'] 

[['', '', None, None, None, None, None, None, None, None, None, ''],
 ['', 'Brød', None, None, None, None, None, None, None, None, None, ''],
 ['', '', None, '', None, None, '', None, None, '', None, ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['', 'Madvare', '', '', 'Lille', '', '', 'Mellem', '', '', 'Stor', ''],
 ['', 'Bagel', '', '', '', '', '', '90', '', '', '', ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  'Bolle, almindelig / fuldkorn – bager/industri',
  '',
  '',
  '40',
  '',
  '',
  '60',
  '',
  '',
  '80',
  ''],
 ['',
  'Bolle, almindelig / fuldkorn – hjemmebagt',
  '',
  '',
  '50',
  '',
  '',
  '70',
  '',
  '',
  '120',
  ''],
 ['', '', '', '', '', '', '', '', '', '', '', ''],
 ['',
  'Burger/hamburger bolle, almindelig/fuldkorn',
  '',
  '',
  '',
  '',
  '',
  '60',
  '',
  '',
  '90',
  ''],
 ['',
  'Ciabattabolle, almindelig/fuldkorn',
  '',
  '',
  '',
  '',
  '',
  '70',
  '',
  '',
  '',
  ''],
 ['', '', '', '', '', '', '', '', '',

In [193]:
import tabulate
df = pd.DataFrame(tables[4]['data'])
print(df.to_markdown())

|    | 0   | 1                      | 2   | 3   | 4      | 5   | 6   | 7        | 8   | 9   | 10      | 11   |
|---:|:----|:-----------------------|:----|:----|:-------|:----|:----|:---------|:----|:----|:--------|:-----|
|  0 |     |                        |     |     |        |     |     |          |     |     |         |      |
|  1 |     | Mel - fortsat          |     |     |        |     |     |          |     |     |         |      |
|  2 |     |                        |     |     |        |     |     |          |     |     |         |      |
|  3 |     | Madvare                |     |     | g / dl |     |     | g / spsk |     |     | g / tsk |      |
|  4 |     | Hvedemel               |     |     | 60     |     |     | 9        |     |     | 3       |      |
|  5 |     | Kartoffelmel           |     |     | 70     |     |     | 11       |     |     | 4       |      |
|  6 |     | Majsmel                |     |     | 65     |     |     | 10       |     |     | 3       |      |
|

# Extraction code

### Remove empty colums and rows

In [194]:
df.replace("", float("NaN"), inplace=True)
df.dropna(how='all', axis=1, inplace=True) #Columns
df.dropna(how = 'all', axis = 0,  inplace = True) #Row
print(df.to_markdown())

|    | 1                      | 4      | 7        | 10      |
|---:|:-----------------------|:-------|:---------|:--------|
|  1 | Mel - fortsat          |        |          |         |
|  3 | Madvare                | g / dl | g / spsk | g / tsk |
|  4 | Hvedemel               | 60     | 9        | 3       |
|  5 | Kartoffelmel           | 70     | 11       | 4       |
|  6 | Majsmel                | 65     | 10       | 3       |
|  7 | Majsstivelse           | 50     | 8        | 3       |
|  8 | Perlebyg               | 70     | 11       | 4       |
|  9 | Quinoa                 | 70     | 11       | 4       |
| 10 | Rasp                   | 45     | 7        | 2       |
| 11 | Ris, brune             | 85     | 13       | 4       |
| 12 | Ris, grød              | 90     | 14       | 5       |
| 13 | Ris, løse (alle slags) | 80     | 12       | 4       |
| 14 | Rismel                 | 65     | 10       | 3       |
| 15 | Rugkerner, hele        | 75     | 11       | 4       |
| 16 | R

/var/folders/95/6_45t3ps6xx37f6psgqx50900000gq/T/ipykernel_28216/378557771.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("", float("NaN"), inplace=True)


### Remove everything above headers

In [195]:
#Find index where "Madvare"-string is present
matching_rows = df[df.apply(lambda row: row.astype(str).str.contains('Madvare', case=False, na=False).any(), axis=1)]

# Get the indices of the matching rows
idx = matching_rows.index.tolist()[0]
print(f"idx: {idx}")
#Extract the cells of this row as these are going to function as header
new_columns = df.loc[int(idx)].to_list()
df.columns = new_columns

#Slice away from "Madvare"-row to the top
df = df.loc[idx+1:]

#df = df[int(idx+1):]
# print(df.to_markdown())

idx: 3


In [196]:
df

,Madvare,g / dl,g / spsk,g / tsk
4,Hvedemel,60,9,3
5,Kartoffelmel,70,11,4
6,Majsmel,65,10,3
7,Majsstivelse,50,8,3
8,Perlebyg,70,11,4
9,Quinoa,70,11,4
10,Rasp,45,7,2
11,"Ris, brune",85,13,4
12,"Ris, grød",90,14,5
13,"Ris, løse (alle slags)",80,12,4


### Refining headers

In [208]:
import re

new_names = []

for i in range(0,len(df.columns.to_list())):
    pattern = r"g\s*/\s*(.*)"

    # Extracting the substring
    match = re.search(pattern, df.columns[i])
    if match: 
        new_names.append(match.group(1))
    else:
        new_names.append(df.columns[i])

df.columns = new_names
df

,Madvare,dl,spsk,tsk
4,Hvedemel,60,9,3
5,Kartoffelmel,70,11,4
6,Majsmel,65,10,3
7,Majsstivelse,50,8,3
8,Perlebyg,70,11,4
9,Quinoa,70,11,4
10,Rasp,45,7,2
11,"Ris, brune",85,13,4
12,"Ris, grød",90,14,5
13,"Ris, løse (alle slags)",80,12,4


In [210]:
df.columns

Index(['Madvare', 'dl', 'spsk', 'tsk'], dtype='object')

In [213]:
melted_df = df.melt(
    id_vars=["Madvare"],
    value_vars=["dl","spsk", "tsk"],
    var_name="Enhed",
    value_name="Konverteringsfaktor"
)
melted_df

,Madvare,Enhed,Konverteringsfaktor
0,Hvedemel,dl,60
1,Kartoffelmel,dl,70
2,Majsmel,dl,65
3,Majsstivelse,dl,50
4,Perlebyg,dl,70
5,Quinoa,dl,70
6,Rasp,dl,45
7,"Ris, brune",dl,85
8,"Ris, grød",dl,90
9,"Ris, løse (alle slags)",dl,80


In [216]:
mylist = [1,2,3,4]

mylist[1:]

[2, 3, 4]